In [1]:
# ✅ Installation des bibliothèques nécessaires (Colab ou Jupyter)
%pip install -U pandas matplotlib seaborn nltk spacy wordcloud unidecode numpy==1.26.4
%pip install -U transformers
%pip install sentence-transformers==2.2.2 huggingface_hub==0.10.1

Note: you may need to restart the kernel to use updated packages.
  Using cached transformers-4.53.3-py3-none-any.whl.metadata (40 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.53.3-py3-none-any.whl (10.8 MB)
   ---------------------------------------- 0.0/558.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/558.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/558.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/558.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/558.7 kB ? eta -:--:--
   ------------------ --------------------- 262.1/558.7 kB ? eta -:--:--
   ------------------ --------------------- 262.1/558.7 kB ? eta -:--:--
   ------------------ --------------------- 262.1/558.7 kB ? eta -:--:--
   -------------------------------------- 558.7/558.7 kB 412.4 kB/s eta 0:00:00
Using cached tokenizers-0.21.2-cp39-abi3-win_amd64.whl (2.5 MB)

  Attemptin

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import re
import nltk
import spacy
from nltk.corpus import stopwords
from unidecode import unidecode

In [3]:
all_reviews2=pd.read_csv("review_cleaned.csv")

In [4]:
all_reviews2['cleaned_lemmatized'].head(5)

0    great host apartment clean everything needed l...
1                 nice place host really nice reactive
2    excellent accommodation new apartment modern c...
3    wow night stay beautiful apartment everything ...
4    bilel responsive message quick clear simple ap...
Name: cleaned_lemmatized, dtype: object

## 🧠 1. Analyse de sentiment avec BERT

Dans cette section, nous utilisons le modèle pré-entraîné `bert-base` via la bibliothèque `transformers` pour détecter le **sentiment** de chaque review nettoyée.  
Chaque texte est tronqué à 512 tokens pour respecter la limite de BERT.

Nous ajoutons une colonne `sentiment_bert` avec la prédiction (`positive`, `negative`, `neutral`).


In [5]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")

def bert_sentiment(text):
    if pd.isnull(text):
        return "neutral"
    result = sentiment_pipeline(text[:512])[0]  # limite à 512 tokens
    return result['label'].lower()

all_reviews2["sentiment_bert"] = all_reviews2["cleaned_lemmatized"].apply(bert_sentiment)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [6]:
all_reviews2["sentiment_bert"]

0        positive
1        positive
2        positive
3        positive
4        positive
           ...   
14624    positive
14625    positive
14626    positive
14627    positive
14628    positive
Name: sentiment_bert, Length: 14629, dtype: object

In [7]:
all_reviews2["sentiment_bert"].value_counts()

sentiment_bert
positive    13813
negative      816
Name: count, dtype: int64

## 📊 2. Scoring des sentiments et agrégation

Nous convertissons les sentiments (`positive`, `neutral`, `negative`) en scores numériques :
- `positive` → 1
- `neutral` → 0
- `negative` → -1

Ensuite, nous calculons la **moyenne des scores** par logement (`id_listing`) et fusionnons ce résultat avec le DataFrame initial.


In [8]:
# Assigner des scores numériques aux sentiments
sentiment_score = {'positive': 1, 'neutral': 0, 'negative': -1}
all_reviews2["sentiment_score"] = all_reviews2["sentiment_bert"].map(sentiment_score)

# Calcul du score moyen par logement
sentiment_moyen_par_listing = all_reviews2.groupby("id_listing")["sentiment_score"].mean().reset_index()
sentiment_moyen_par_listing.rename(columns={"sentiment_score": "sentiment_moyen"}, inplace=True)

# Fusionner avec reviews
all_reviews2 = all_reviews2.merge(sentiment_moyen_par_listing, on="id_listing", how="left")


## ✅ 3. Sélection des logements bien notés

Nous filtrons uniquement les logements avec un **score de sentiment moyen > 0.2**, ce qui indique des avis globalement positifs.


In [9]:
# Option : filtrer uniquement les logements bien notés
bons_logements = sentiment_moyen_par_listing[sentiment_moyen_par_listing["sentiment_moyen"] > 0.2]["id_listing"]


In [10]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [11]:
import folium
from folium.plugins import HeatMap

# Filtrer uniquement les avis positifs
positifs = all_reviews2[all_reviews2["sentiment_bert"] == "positive"]

# Assurer qu’on a bien des coordonnées
positifs = positifs.dropna(subset=["coordinates/latitude", "coordinates/longitude"])

# Créer carte centrée sur la Tunisie (approximativement)
carte = folium.Map(location=[34.0, 10.0], zoom_start=6)

# Ajouter une heatmap
heat_data = list(zip(positifs["coordinates/latitude"], positifs["coordinates/longitude"]))
HeatMap(heat_data).add_to(carte)

carte.save("avis_positifs_heatmap.html")


## 🧬 4. Représentation TF-IDF et Similarité Cosinus

Nous regroupons tous les avis par logement, puis appliquons **TF-IDF** pour transformer le texte en vecteurs.  
Enfin, nous calculons la **matrice de similarité cosinus** entre logements.

Cela servira plus tard à faire de la recommandation de logements similaires selon le contenu textuel des reviews.


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Grouper les textes par logement
df_grouped = all_reviews2.groupby("id_listing")["cleaned_lemmatized"].apply(lambda x: " ".join(x)).reset_index()

# TF-IDF
tfidf = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf.fit_transform(df_grouped["cleaned_lemmatized"])

# Similarité
similarity_matrix = cosine_similarity(X_tfidf)

# Mapping id <-> index
id_to_index = {id_: i for i, id_ in enumerate(df_grouped["id_listing"])}


## 🧾 5. Construction de métadonnées par logement

Nous agrégeons plusieurs indicateurs par logement (`id_listing`) pour enrichir notre système de recommandation :
- **Moyenne du score de sentiment** (`sentiment_score`)
- **Moyenne de la note review** (`rating_review`)
- **Moyenne de la précision** (`rating/accuracy`)

Nous les combinons dans un seul DataFrame `metadata`, qui servira plus tard à des approches hybrides (collaboratif + contenu).


In [13]:
# Moyenne du sentiment
sentiment_moyen = all_reviews2.groupby("id_listing")["sentiment_score"].mean()
# Moyenne des notes review
rating_review_moyen = all_reviews2.groupby("id_listing")["rating_review"].mean()
# Moyenne des rating/accuracy
accuracy_moyen = all_reviews2.groupby("id_listing")["rating/accuracy"].mean()

# Fusionner dans un seul DataFrame
metadata = pd.DataFrame({
    "sentiment_moyen": sentiment_moyen,
    "rating_review_moyen": rating_review_moyen,
    "accuracy_moyen": accuracy_moyen
}).reset_index()


## 🤝 6. Recommandation hybride pondérée avec explication

Cette fonction `recommander_logements_expliques()` permet de recommander des **logements similaires** à un `id_listing` donné, en combinant :

- 🔍 **Similarité textuelle** (TF-IDF des reviews),
- 💬 **Sentiment moyen** des avis,
- 📊 **Note moyenne** et **précision** (`accuracy`).

Chaque facteur est pondéré :
- `alpha` : poids de la similarité textuelle (par défaut : 0.5)
- `beta` : poids du sentiment moyen (par défaut : 0.3)
- `gamma` : poids de la moyenne des notes (`rating_review` et `accuracy`) (par défaut : 0.2)

La sortie affiche les recommandations avec :
- Le **titre**, la **ville**, et une partie de la **description**,
- Les **scores pondérés** utilisés,
- Quelques **exemples de reviews positives** du logement recommandé.

Cette approche permet une **recommandation transparente** et compréhensible pour l’utilisateur final.


In [14]:
def recommander_logements_expliques(id_listing, top_n=5, alpha=0.3, beta=0.5, gamma=0.2):
    """
    Recommande des logements similaires avec explications
    """
    idx = id_to_index.get(id_listing)
    if idx is None:
        print("ID inconnu.")
        return []

    sim_scores = list(enumerate(similarity_matrix[idx]))

    results = []
    for i, sim in sim_scores:
        id_candidat = df_grouped.iloc[i]["id_listing"]
        if id_candidat == id_listing:
            continue

        sent = metadata.loc[metadata["id_listing"] == id_candidat, "sentiment_moyen"].values[0]
        rev = metadata.loc[metadata["id_listing"] == id_candidat, "rating_review_moyen"].values[0]
        acc = metadata.loc[metadata["id_listing"] == id_candidat, "accuracy_moyen"].values[0]

        score_final = (
            alpha * sim +
            beta * sent +
            gamma * ((rev + acc) / 10)  # normalize 0–1
        )

        results.append({
            "id_listing": id_candidat,
            "score": score_final,
            "similarity_text": sim,
            "sentiment": sent,
            "rating_review": rev,
            "accuracy": acc
        })

    # Trier les résultats
    top_results = sorted(results, key=lambda x: x["score"], reverse=True)[:top_n]

    # Affichage
    for res in top_results:
        id_l = res["id_listing"]
        infos = all_reviews2[all_reviews2["id_listing"] == id_l].iloc[0]

        print("🏠 Maison d'hôte :", infos["title"])
        print("📍 Ville :", infos["city_listing"])
        print("📝 Description :", infos["description"][:300], "...")
        print("⭐ Score global pondéré :", round(res["score"], 3))
        print("🔍 Similarité texte :", round(res["similarity_text"], 3))
        print("💬 Sentiment moyen :", round(res["sentiment"], 3))
        print("📊 Note review moyenne :", round(res["rating_review"], 2))
        print("🎯 Accuracy moyenne :", round(res["accuracy"], 2))

        print("💬 Exemple de review :")
        example_reviews = all_reviews2[(all_reviews2["id_listing"] == id_l) & (all_reviews2["sentiment_bert"] == "positive")]["localizedText"].dropna().tolist()
        for r in example_reviews[:2]:  # max 2 reviews
            print("   •", r[:200].strip(), "...")
        print("-" * 80)

    return [r["id_listing"] for r in top_results]



In [15]:
all_reviews2['id_listing']

0        1304393643592186021
1        1304393643592186021
2        1304393643592186021
3        1304393643592186021
4        1304393643592186021
                ...         
14624                5734121
14625                5734121
14626                5734121
14627                5734121
14628                5734121
Name: id_listing, Length: 14629, dtype: int64

In [16]:
recommander_logements_expliques(id_listing=5734121, top_n=3)


🏠 Maison d'hôte : Dar Babel - Un Riad Djerbien au Coeur d 'Erriadh
📍 Ville : Jerba
📝 Description : Dar Babel offers you the unique opportunity to stay in the enchanting setting of an authentic Djerbian Riad. A haven of peace located in the middle of a centuries-old village (Erriadh) of this island (Djerba ) " where the air is so mild that it prevents it from getting away " (Flaubert).Dar Babel is ...
⭐ Score global pondéré : 0.959
🔍 Similarité texte : 0.864
💬 Sentiment moyen : 1.0
📊 Note review moyenne : 5.0
🎯 Accuracy moyenne : 5.0
💬 Exemple de review :
   • we loved staying here the decor and feel of the place give a very authentic tunisian experience zouheir was very responsive and helpful giving good suggestions for the island rebeh was an amazing host ...
   • the stay at dar babel was so nice that it is hard to describe i felt very comfortable the house is beautiful and rebeh lives next door she is a wonderful person who is always willing to answer questio ...
-------------------

[36228466, 18847867, 7998836]

##  Recommandation avec BERT Embeddings (remplace TF-IDF)

In [17]:
# ✅ Réinstallation compatible (exécute dans une cellule)
!pip install -U sentence-transformers==2.2.2
!pip install -U huggingface_hub==0.10.1


## 🧠 7. Similarité sémantique avec Sentence-BERT

Nous remplaçons la représentation TF-IDF par des **embeddings BERT** générés avec `SentenceTransformer` pour capturer la **sémantique globale** des textes.

### Modèle utilisé :
- [`all-MiniLM-L6-v2`](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) : rapide, multilingue et bien adapté à la similarité de phrases.

### Étapes :
1. Les textes regroupés par logement sont vectorisés via BERT.
2. On calcule la **matrice de similarité cosinus** sur ces embeddings.
3. Cette nouvelle matrice `similarity_matrix_bert` remplace la précédente (`similarity_matrix` basée sur TF-IDF).


In [18]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ⚙️ Charger modèle BERT optimisé pour sentence similarity
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

# 🧹 Texte à encoder (nettoyé)
texts = df_grouped["cleaned_lemmatized"].fillna("").tolist()

# 🧠 Obtenir embeddings
embeddings = bert_model.encode(texts, convert_to_tensor=True)

# 📐 Matrice de similarité
similarity_matrix_bert = cosine_similarity(embeddings.cpu().numpy())


## 📈 8. Optimisation de l'agrégation des métadonnées

Nous utilisons `.groupby().agg()` pour agréger plusieurs indicateurs par `id_listing` de manière plus compacte :

- **Sentiment moyen** : proportion d’avis positifs (`sentiment_bert == "positive"`)
- **Note moyenne** (`rating_review`)
- **Précision moyenne** (`rating/accuracy`)

Le résultat est un DataFrame `metadata` prêt à être utilisé pour la recommandation hybride.


In [19]:
metadata = all_reviews2.groupby("id_listing").agg({
    "sentiment_bert": lambda x: (x == "positive").mean(),
    "rating_review": "mean",
    "rating/accuracy": "mean"
}).reset_index().rename(columns={
    "sentiment_bert": "sentiment_moyen",
    "rating_review": "rating_review_moyen",
    "rating/accuracy": "accuracy_moyen"
})


✅ Étape 2 : Créer le dictionnaire id_to_index

In [20]:
id_to_index = {row["id_listing"]: idx for idx, row in df_grouped.iterrows()}


## 🤖 9. Recommandation hybride finale (BERT + métadonnées)

Nous utilisons une version finale de la fonction `recommander_logements_expliques()` qui s’appuie sur :

- 🔍 **Similarité sémantique des avis** via Sentence-BERT (`all-MiniLM-L6-v2`)
- 💬 **Sentiment moyen** des reviews (proportion de positifs)
- 🗣️ **Note moyenne des reviews**
- 🎯 **Précision** (`rating/accuracy`)

Les pondérations sont ajustables :
- `alpha` → similarité textuelle (par défaut 0.5)
- `beta` → sentiment (par défaut 0.3)
- `gamma` → note + accuracy (par défaut 0.2)

La fonction retourne les logements les plus pertinents avec :
- **Scores détaillés**
- **Infos principales**
- **2 exemples de reviews positives**


In [21]:
def recommander_logements_expliques(id_listing, top_n=5, alpha=0.5, beta=0.3, gamma=0.2):
    """
    Recommande des logements similaires à un logement donné
    en combinant similarité BERT, sentiment, review rating, et accuracy.
    """
    idx = id_to_index.get(id_listing)
    if idx is None:
        print("❌ ID de logement inconnu.")
        return []

    sim_scores = list(enumerate(similarity_matrix_bert[idx]))
    results = []

    for i, sim in sim_scores:
        id_candidat = df_grouped.iloc[i]["id_listing"]
        if id_candidat == id_listing:
            continue

        try:
            sent = metadata.loc[metadata["id_listing"] == id_candidat, "sentiment_moyen"].values[0]
            rev = metadata.loc[metadata["id_listing"] == id_candidat, "rating_review_moyen"].values[0]
            acc = metadata.loc[metadata["id_listing"] == id_candidat, "accuracy_moyen"].values[0]
        except IndexError:
            continue  # Sauter si valeurs manquantes

        score_final = (
            alpha * sim +
            beta * sent +
            gamma * ((rev + acc) / 10)  # Normalisé 0–1
        )

        results.append({
            "id_listing": id_candidat,
            "score": score_final,
            "similarity_text": sim,
            "sentiment": sent,
            "rating_review": rev,
            "accuracy": acc
        })

    top_results = sorted(results, key=lambda x: x["score"], reverse=True)[:top_n]

    for res in top_results:
        id_l = res["id_listing"]
        infos = all_reviews2[all_reviews2["id_listing"] == id_l].iloc[0]

        print("🏡 Maison d'hôte :", infos["title"])
        print("📍 Ville :", infos["city_listing"])
        print("📝 Description :", infos["description"][:250].strip(), "...")
        print(f"⭐ Score global : {res['score']:.3f}")
        print(f"🔍 Similarité texte (BERT) : {res['similarity_text']:.3f}")
        print(f"💬 Sentiment moyen : {res['sentiment']:.3f}")
        print(f"🗣️ Note review moyenne : {res['rating_review']:.2f}")
        print(f"🎯 Accuracy moyenne : {res['accuracy']:.2f}")

        print("💬 Exemple de review positive :")
        reviews_pos = all_reviews2[
            (all_reviews2["id_listing"] == id_l) &
            (all_reviews2["sentiment_bert"] == "positive")
        ]["localizedText"].dropna().tolist()

        for r in reviews_pos[:2]:  # max 2 reviews
            print("   •", r[:200].strip(), "...")

        print("-" * 100)

    return [r["id_listing"] for r in top_results]


In [22]:
# from IPython.display import display, HTML

# def recommander_logements_expliques_visuel(id_listing, top_n=5, alpha=0.5, beta=0.3, gamma=0.2):
#     idx = id_to_index.get(id_listing)
#     if idx is None:
#         display(HTML("<p style='color:red;'>❌ ID de logement inconnu.</p>"))
#         return []

#     sim_scores = list(enumerate(similarity_matrix_bert[idx]))
#     results = []

#     for i, sim in sim_scores:
#         id_candidat = df_grouped.iloc[i]["id_listing"]
#         if id_candidat == id_listing:
#             continue

#         try:
#             sent = metadata.loc[metadata["id_listing"] == id_candidat, "sentiment_moyen"].values[0]
#             rev = metadata.loc[metadata["id_listing"] == id_candidat, "rating_review_moyen"].values[0]
#             acc = metadata.loc[metadata["id_listing"] == id_candidat, "accuracy_moyen"].values[0]
#         except IndexError:
#             continue

#         score_final = (
#             alpha * sim +
#             beta * sent +
#             gamma * ((rev + acc) / 10)
#         )

#         results.append({
#             "id_listing": id_candidat,
#             "score": score_final,
#             "similarity_text": sim,
#             "sentiment": sent,
#             "rating_review": rev,
#             "accuracy": acc
#         })

#     top_results = sorted(results, key=lambda x: x["score"], reverse=True)[:top_n]

#     html_content = "<div style='font-family:Arial, sans-serif;'>"
#     for res in top_results:
#         id_l = res["id_listing"]
#         infos = all_reviews2[all_reviews2["id_listing"] == id_l].iloc[0]

#         html_content += f"""
#         <div style='border:1px solid #ddd; padding:15px; margin-bottom:20px; border-radius:8px; background:#fafafa;'>
#             <h2 style='color:#2a9d8f;'>🏡 {infos["title"]} <small style='color:#555;'>({infos["city_listing"]})</small></h2>
#             <p style='font-style:italic; color:#555;'>{infos["description"][:250].strip()}...</p>
#             <p>
#                 <b>⭐ Score global :</b> {res['score']:.3f} &nbsp;&nbsp;
#                 <b>🔍 Similarité texte (BERT) :</b> {res['similarity_text']:.3f} &nbsp;&nbsp;
#                 <b>💬 Sentiment moyen :</b> {res['sentiment']:.3f} <br>
#                 <b>🗣️ Note review moyenne :</b> {res['rating_review']:.2f} &nbsp;&nbsp;
#                 <b>🎯 Accuracy moyenne :</b> {res['accuracy']:.2f}
#             </p>
#             <div>
#                 <b>💬 Exemple(s) de review positive :</b>
#                 <ul style='color:#264653;'>
#         """

#         reviews_pos = all_reviews2[
#             (all_reviews2["id_listing"] == id_l) &
#             (all_reviews2["sentiment_bert"] == "positive")
#         ]["localizedText"].dropna().tolist()

#         for r in reviews_pos[:2]:
#             safe_review = r.replace('<', '&lt;').replace('>', '&gt;')
#             html_content += f"<li>{safe_review[:200].strip()}...</li>"

#         html_content += """
#                 </ul>
#             </div>
#         </div>
#         """

#     html_content += "</div>"

#     display(HTML(html_content))

#     return [r["id_listing"] for r in top_results]


In [23]:
# Choisir un logement à tester (ex: un des ID dans df_grouped)
id_test = df_grouped["id_listing"].iloc[0]
recommander_logements_expliques(id_test)


🏡 Maison d'hôte : Dar Sabri, Ebène suite for 2 people
📍 Ville : Hammamet
📝 Description : Welcome to Dar Sabri. An unprecedented place in Tunisia, designed in the love of authenticity and refinement.The spaceLocated upstairs, impressive modern, the Ebène suite adorns with gray, black and white tones. This suite also has a Bang & Olufsen f ...
⭐ Score global : 0.899
🔍 Similarité texte (BERT) : 0.830
💬 Sentiment moyen : 1.000
🗣️ Note review moyenne : 4.60
🎯 Accuracy moyenne : 4.60
💬 Exemple de review positive :
   • was nice being there room is good pool was perfect cool helpful stuff ...
   • a harbor of peace within the medina sabri has succeeded in his bet combining tradition and modernity in a guest house like the riads of marrakech hospitality is waiting for you and starts with a welco ...
----------------------------------------------------------------------------------------------------
🏡 Maison d'hôte : Galaxy Dorret El Hammamet Hotel
📍 Ville : Hammamet
📝 Description : For your va

[2741302, 1204226718042735221, 37448008, 24342792, 1163131993955627305]

✅ 1. Widget interactif pour tester la recommandation

In [24]:
all_reviews_new=all_reviews2.to_csv('all_reviews_final.csv', index=False)


In [25]:
reviews=pd.read_csv('all_reviews_final.csv')

In [26]:
reviews.columns

Index(['id_review', 'text', 'localizedText', 'rating_review', 'createdAt',
       'language', 'reviewer/id', 'id_listing', 'title', 'description',
       'city_listing', 'rating_listing', 'rating/cleanliness',
       'rating/accuracy', 'rating/checking', 'rating/communication',
       'rating/value', 'rating/location', 'rating/guestSatisfaction',
       'rating/reviewsCount', 'price/price', 'price/label',
       'coordinates/latitude', 'coordinates/longitude', 'tokens',
       'cleaned_no_stopwords', 'orthographic_anomalies',
       'cleaned_no_stopwords_no_anomalies', 'rare_words', 'has_rare_words',
       'cleaned_text', 'cleaned_lemmatized', 'sentiment_bert',
       'sentiment_score', 'sentiment_moyen'],
      dtype='object')

In [27]:
pip install ipywidgets folium


Note: you may need to restart the kernel to use updated packages.


## 📊 10. Ratio de reviews positives & enrichissement des métadonnées

Nous ajoutons une nouvelle métrique : le **ratio de reviews positives** par logement.

Ensuite, nous fusionnons cette métrique avec le DataFrame `metadata` pour obtenir un DataFrame enrichi `df_scores` qui sera utilisé dans une approche **hybride sans texte** (pas de similarité TF-IDF ni BERT).


In [28]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.notebook import tqdm


In [29]:
# Calcul du ratio de reviews positives
review_stats = all_reviews2.groupby("id_listing").agg(
    nb_reviews=("sentiment_bert", "count"),
    nb_pos=("sentiment_bert", lambda x: (x == "positive").sum())
).reset_index()

review_stats["positive_ratio"] = review_stats["nb_pos"] / review_stats["nb_reviews"]

# Fusion avec metadata
df_scores = metadata.merge(review_stats[["id_listing", "positive_ratio"]], on="id_listing", how="left")


## 🧮 11. Recommandation hybride sans description

Dans cette version, nous recommandons des logements uniquement à partir de **métadonnées numériques** :

- 💬 Sentiment moyen
- ⭐ Note moyenne
- 🎯 Accuracy moyenne
- 👍 Ratio de reviews positives

Le score final est calculé comme :


In [30]:
alpha, beta, gamma, delta = 0.3, 0.3, 0.2, 0.2  # pondérations

def score_hybride_sans_description(id_source, top_n=10):
    if id_source not in id_to_index:
        print("❌ ID source inconnu.")
        return []

    results = []
    for _, row in df_scores.iterrows():
        id_cand = row["id_listing"]
        if id_cand == id_source:
            continue

        s_moy = row["sentiment_moyen"]
        note = row["rating_review_moyen"]
        acc = row["accuracy_moyen"]
        pos_ratio = row["positive_ratio"]

        # Calcul du score sans similarité description
        score = (
            beta * s_moy +
            gamma * ((note + acc) / 20) +
            delta * pos_ratio
        )

        results.append({
            "id_listing": id_cand,
            "sentiment": s_moy,
            "note": note,
            "accuracy": acc,
            "positive_ratio": pos_ratio,
            "score": score
        })

    top_results = sorted(results, key=lambda x: x["score"], reverse=True)[:top_n]
    return top_results



## 🖥️ 12. Affichage des recommandations (métadonnées uniquement)

Cette fonction `afficher_recommandations()` affiche les **recommandations générées à partir des métadonnées uniquement**, sans faire appel à une similarité textuelle.

Pour chaque logement recommandé, on affiche :
- 📌 Ville, titre, score global
- 💬 Sentiment moyen
- 👍 Ratio de reviews positives
- ⭐ Note & 🎯 Accuracy
- 💬 2 exemples de reviews positives


In [31]:
def afficher_recommandations(id_source, top_n=10):
    recommandations = score_hybride_sans_description(id_source, top_n=top_n)

    for res in recommandations:
        infos = all_reviews2[all_reviews2["id_listing"] == res["id_listing"]].iloc[0]

        print(f"🏡 {infos['title']} ({infos['city_listing']})")
        # Suppression de la similarité texte car elle n’est plus calculée
        # print(f"🔍 Similarité texte : {res['similarity']:.3f}")  

        print(f"💬 Sentiment moyen : {res['sentiment']:.3f}")
        print(f"👍 Reviews positives : {res['positive_ratio']:.2%}")
        print(f"⭐ Note : {res['note']:.2f} | 🎯 Accuracy : {res['accuracy']:.2f}")
        print(f"🔝 Score global : {res['score']:.4f}")

        pos_reviews = all_reviews2[
            (all_reviews2["id_listing"] == res["id_listing"]) &
            (all_reviews2["sentiment_bert"] == "positive")
        ]["localizedText"].dropna().tolist()

        print("💬 Extrait(s) de review positive :")
        for r in pos_reviews[:2]:
            print("   •", r[:200].strip(), "...")
        print("-" * 80)


## ▶️ Étape 5 : Tester sur un logement

Changer l’ID ci-dessous pour tester un autre logement.


In [32]:
id_exemple = all_reviews2["id_listing"].iloc[10]  # ou un id connu
afficher_recommandations(id_exemple, top_n=10)


🏡 Sidi Bou Said-style villa with pool (Hammamet)
💬 Sentiment moyen : 1.000
👍 Reviews positives : 100.00%
⭐ Note : 5.00 | 🎯 Accuracy : 5.00
🔝 Score global : 0.6000
💬 Extrait(s) de review positive :
   • great stay in a pleasant fully equipped house until hygiene product the house was spotless when we arrived we just had to drop off our luggage ideally located the house is near the medina of the beach ...
   • what a fantastic clean villa had a great family vacation i fell in love with the villa ahmed the owner is a great guy very helpful arrive at 2150 and waited for us until 2250 to checkin the photos don ...
--------------------------------------------------------------------------------
🏡 Sun beach and fun! (Hammamet)
💬 Sentiment moyen : 1.000
👍 Reviews positives : 100.00%
⭐ Note : 5.00 | 🎯 Accuracy : 5.00
🔝 Score global : 0.6000
💬 Extrait(s) de review positive :
   • clean and well located accommodation close to restaurants shops bars beachmolka is very responsive and accommodatingw

# ⚖️ Résumé Comparatif des Systèmes de Recommandation

| Système                            | Texte | Notes | Sentiment | Robuste | Complexité |
|-----------------------------------|:-----:|:-----:|:---------:|:-------:|:----------:|
| Métadonnées simples               |   ❌   |  ✅   |    ✅     |   ✅    |    🔹 Faible   |
| TF-IDF                            |   ✅   |  ❌   |    ❌     |  ⚠️ Moy  |   🔸 Moyenne   |
| BERT                              |  ✅✅  |  ❌   |    ❌     |  ⚠️ Moy  |    🔸 Haute    |
| Hybride (BERT + Notes + Sentiment)|  ✅✅  |  ✅   |    ✅     |   ✅    |    🔶 Élevée   |
| Reviews uniquement                |   ❌   |  ✅   |    ✅     |   ✅✅   |    🔹 Faible   |

---

**Conseils :**

- Utiliser le système **BERT + métadonnées** si les descriptions sont bien renseignées et de bonne qualité.
- Utiliser le système **basé uniquement sur les reviews** si les descriptions sont absentes, incomplètes ou peu fiables.
- Il est possible de **combiner les deux approches** pour bénéficier à la fois de la richesse sémantique et de l’expérience utilisateur.
